In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.datasets import load_boston, load_iris

from sklearn.linear_model import Ridge,Lasso,ElasticNet, LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import StandardScaler

from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

import mglearn
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.family']='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] = False

import warnings
warnings.simplefilter('ignore')

In [2]:
train_df = pd.read_excel('./data4/hyundaiCar.xlsx', sheet_name='train')
test_df = pd.read_excel('./data4/hyundaiCar.xlsx', sheet_name='test')

In [3]:
x_train, y_train = train_df.iloc[:, 1:], train_df['가격']
x_test, y_test = test_df.iloc[:, 1:], test_df['가격']

연습문제.

라벨인코딩을 이용하여
2015, 준중형, 12.3, 204, 27,가솔린,0,1591,1300,자동
자동차가격을 예측하시요

In [4]:
x_train

,년식,종류,연비,마력,토크,연료,하이브리드,배기량,중량,변속기
0,2015,준중형,11.8,172,21.0,가솔린,0,1999,1300,자동
1,2015,준중형,12.3,204,27.0,가솔린,0,1591,1300,자동
2,2015,소형,15.0,100,13.6,가솔린,0,1368,1035,수동
3,2014,소형,14.0,140,17.0,가솔린,0,1591,1090,자동
4,2015,대형,9.6,175,46.0,디젤,0,2497,1990,자동
...,...,...,...,...,...,...,...,...,...,...
66,2015,중형,8.5,290,34.8,가솔린,0,3342,1901,자동
67,2012,소형,13.3,108,13.9,가솔린,0,1396,1040,자동
68,2015,준중형,12.8,186,41.0,디젤,0,1995,1665,자동
69,2015,중형,17.7,156,19.3,가솔린,1,1999,1585,자동


In [5]:
x_test.head()

,년식,종류,연비,마력,토크,연료,하이브리드,배기량,중량,변속기
0,2015,대형,6.8,159,23.0,LPG,0,2359,1935,수동
1,2012,소형,13.3,108,13.9,가솔린,0,1396,1035,자동
2,2015,중형,14.4,184,41.0,디젤,0,1995,1792,자동
3,2015,대형,10.9,175,46.0,디젤,0,2497,2210,수동
4,2015,대형,6.4,159,23.0,LPG,0,2359,1935,자동


In [6]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
encoder.fit(x_train[['종류', '연료', '변속기']])
x_train[['종류', '연료', '변속기']] = encoder.transform(x_train[['종류', '연료', '변속기']])


In [7]:
print(x_train.종류.unique(), x_test.종류.unique())

[2. 1. 0. 3.] ['대형' '소형' '중형' '준중형']


In [8]:
x_test[['종류', '연료', '변속기']] = encoder.transform(x_test[['종류', '연료', '변속기']])

In [9]:
from sklearn.pipeline import Pipeline
model_elasticnet = Pipeline([('scl', StandardScaler()), ('clf', ElasticNet())])
param_value = {'clf__alpha':[0.001, 0.01, 1, 2, 3, 4, 10, 100],'clf__l1_ratio':[0, 0.25, 0.5, 0.75, 1] }
gridSearch = GridSearchCV(model_elasticnet, param_grid=param_value, cv=5, verbose=1, scoring='r2')
gridSearch.fit(x_train, y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scl', StandardScaler()),
                                       ('clf', ElasticNet())]),
             param_grid={'clf__alpha': [0.001, 0.01, 1, 2, 3, 4, 10, 100],
                         'clf__l1_ratio': [0, 0.25, 0.5, 0.75, 1]},
             scoring='r2', verbose=1)

In [10]:
gridSearch.best_params_

{'clf__alpha': 1, 'clf__l1_ratio': 0.75}

In [11]:
gridSearch.best_score_

0.7311557986127946

In [12]:
gridSearch.best_estimator_.score(x_test, y_test)

0.6862743665390543

In [13]:
test = pd.DataFrame([[2015, '준중형', 12.3, 204, 27,'가솔린',0,1591,1300,'자동']], columns = x_train.columns)
test[['종류', '연료', '변속기']] = encoder.transform(test[['종류', '연료', '변속기']])
test

,년식,종류,연비,마력,토크,연료,하이브리드,배기량,중량,변속기
0,2015,2.0,12.3,204,27,1.0,0,1591,1300,1.0


In [14]:
gridSearch.best_estimator_.predict(test)

array([2413.62771031])